kernel moscot_env

# Visualize MultiVI on all PCWs

## 0. Set up environment

In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import mudata as md
import muon
import numpy as np
import scanpy as sc
import scvi
import seaborn as sns
import pandas as pd
import anndata as ad

In [ ]:
outdir = f"/work/DevM_analysis/01.annotation/10.integration_joint_clean_MultiVI/"

#### 0.1. Load mvi

In [ ]:
mvi = scvi.model.MULTIVI.load(f"{outdir}/data/model_multiVI_FL_allPCW/")

In [ ]:
mvi

#### 0.2. Load mudata

In [ ]:
mudata_multiome = muon.read("/work/DevM_analysis/01.annotation/10.integration_joint_clean/data/FL_wnn_clustered.v01.h5mu")

In [ ]:
mudata_multiome

In [ ]:
hscs_sub = pd.read_csv("/work/DevM_analysis/01.annotation/11.subclustering/HSC/data/FL_wnn_cellmeta.v00.csv")

In [ ]:
hscs_sub = hscs_sub.set_index("Unnamed: 0")
hscs_sub["leiden_wnn_0.3"]

In [ ]:
mudata_multiome.obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"] = mudata_multiome.obs["anno_wnn_v51"].astype("str").copy()

In [ ]:
mudata_multiome.obs.loc[hscs_sub.index[hscs_sub.index.isin(mudata_multiome.obs_names)], "anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"]="HSC-" + hscs_sub.loc[hscs_sub.index.isin(mudata_multiome.obs_names), "leiden_wnn_0.3"].astype("str")

In [ ]:
annot2col = {
  "HSC" : "#E41A1C",
  "HSC-0" : "#fee5d9",
  "HSC-1" : "#fcae91",
  "HSC-2" : "#fb6a4a",
  "HSC-3" : "#de2d26",
  "HSC-4" : "#a50f15",
  "GP" : "#E0FFFF",
  "Granulocyte" : "#B3CDE3",
  "MEMP-t" : "#E6AB02",
  "MEMP" : "#FF7F00",
  "MEP" : "#CD661D",
  "MEMP-Mast-Ery" : "#FDCDAC",
  "MEMP-Ery" : "#E9967A",
  "Early-Ery" : "#CD5555",
  "Late-Ery" : "#8B0000",
  "MEMP-MK" : "#663C1F",
  "MK" : "#40E0D0",
  "MastP-t" : "#1E90FF",
  "MastP" : "#1F78B4",
  "Mast" : "#253494",
  "MDP" : "#E6F5C9",
  "Monocyte" : "#005A32",
  "Kupffer" : "#00EE00",
  "cDC1" : "#B3DE69",
  "cDC2" : "#ADFF2F",
  "pDC" : "#4DAF4A",
  "ASDC" : "#CDC673",
  "LMPP" : "#FFF2AE",
  "LP" : "#FFD92F",
  "Cycling-LP" : "#FFFF33",
  "PreProB" : "#FFF0F5",
  "ProB-1" : "#FFB5C5",
  "ProB-2" : "#E78AC3",
  "Large-PreB" : "#CD1076",
  "Small-PreB" : "#FF3E96",
  "IM-B" : "#FF00FF",
  "NK" : "#A020F0",
  "ILCP" : "#49006A",
  "T" : "#984EA3",
  "Hepatocyte" : "#666666",
  "Endothelia" : "#000000",
}

In [ ]:
order = []
for name, value in annot2col.items():
    order.append(name)

In [ ]:
o = []
for i in order:
    if i in mudata_multiome.obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"].astype("category").cat.categories.values:
        o.append(i)

In [ ]:
mudata_multiome.obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"] = mudata_multiome.obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"].astype("category")
mudata_multiome.obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"] = mudata_multiome.obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"].cat.reorder_categories(o)

## 1. Process

In [ ]:
mudata_multiome.obsm["X_multiVI"] = mvi.get_latent_representation()

In [ ]:
sc.pp.neighbors(mudata_multiome, use_rep = "X_multiVI")
sc.tl.umap(mudata_multiome)

In [ ]:
mudata_multiome.obsm["X_umap_multiVI"] = mudata_multiome.obsm["X_umap"].copy()

## 2. Plot UMAPs

In [ ]:
#sc.set_figure_params(dpi=1000, dpi_save=1000, transparent = True, figsize = (4.6154,3.84615))

In [ ]:
sc.set_figure_params(dpi=100, dpi_save=150, transparent = True, figsize = (4.6154,3.84615))

In [ ]:
# saved as plot/FL_UMAP_allPCW_multiVI_sampleID(_noLeg).png
muon.pl.embedding(
    mudata_multiome,
    color=["sampleID"],
    size = 0.7,
    ncols=1,
    basis="umap_multiVI",
    frameon=False,
    #save = "FL_UMAP_allPCW_multiVI_sampleID.png",
    #legend_loc = None
)

In [ ]:
age2col = {"5" : "#FFBF00", 
           "6" : "#ff986d", 
           "7" : "#ff8782",
           "8" : "#ff7b9b",
           "9" : "#f176b4",
           "10" : "#d97fce",
           "11" : "#b989e2",
           "12" : "#9094ed",
           "13" : "#60a4f2",
           "14" : "#32b1eb",
           "15" : "#29badb",
           "16" : "#4bc0c8",
           "17" : "#8dd7db",
           "18" : "#c0dedf",
           "Missing" : "#9f9f9f"}

In [ ]:
# saved as plot/FL_UMAP_allPCW_multiVI_colorPCW(_noLeg).png
muon.pl.embedding(
    mudata_multiome,
    color=["PCW"],
    ncols=1,
    size = 0.7,
    basis="umap_multiVI",
    frameon=False,
    palette = age2col, 
    #save = "FL_UMAP_allPCW_multiVI_colorPCW_noLeg.png", 
   #legend_loc = None
)

In [ ]:
phase2color = {'G1':"tab:blue", 'S':"tab:orange", 'G2M':"tab:green"}

In [ ]:
# saved as plot/FL_UMAP_allPCW_multiVI_rna-Phase(_noLeg).png
muon.pl.embedding(
    mudata_multiome,
    color=["rna:Phase"],
    size = 0.7,
    ncols=1,
    basis="umap_multiVI",
    frameon=False,
    palette = phase2color,
    #save = "FL_UMAP_allPCW_multiVI_rna-Phase_noLeg.png",
    #legend_loc = None
)

In [ ]:
# saved as plot/FL_UMAP_allPCW_multiVI_anno-wnn-v51(_noLeg).png
muon.pl.embedding(
    mudata_multiome,
    color=["anno_wnn_v51"],
    ncols=1,
    basis="umap_multiVI",
    size = 0.7,
    frameon=False,
    palette = annot2col,
    #save = "FL_UMAP_allPCW_multiVI_anno-wnn-v51_noLeg.png",
    #legend_loc = None
)

In [ ]:
muon.pl.embedding(
    mudata_multiome,
    color=["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"],
    ncols=1,
    basis="umap_multiVI",
    frameon=False,
    palette = annot2col
)

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(9, 9))
plt.subplots_adjust(wspace=0.5)
muon.pl.embedding(mudata_multiome, color=["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"], 
                  groups=["HSC-0"], ax=axs[0][0], show = False, basis = "X_umap_multiVI")
muon.pl.embedding(mudata_multiome, color=["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"], 
                  groups=["HSC-1"], ax=axs[1][0], show = False, basis = "X_umap_multiVI")
muon.pl.embedding(mudata_multiome, color=["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"], 
                  groups=["HSC-2"], ax=axs[1][1], show = False, basis = "X_umap_multiVI")
muon.pl.embedding(mudata_multiome, color=["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"], 
                  groups=["HSC-3"], ax=axs[2][0], show = False, basis = "X_umap_multiVI")
muon.pl.embedding(mudata_multiome, color=["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"], 
                  groups=["HSC-4"], ax=axs[2][1], show = False, basis = "X_umap_multiVI")

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(9, 9))
plt.subplots_adjust(wspace=0.5)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MEMP"], ax=axs[0][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MEMP-Ery"], ax=axs[0][1], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MEMP-MK"], ax=axs[1][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MEMP-Mast-Ery"], ax=axs[1][1], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MEP"], ax=axs[2][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MEMP-t"], ax=axs[2][1], show = False)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))
plt.subplots_adjust(wspace=0.5)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["Mast"], ax=axs[0][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MastP"], ax=axs[1][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["MastP-t"], ax=axs[1][1], show = False)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 6))
plt.subplots_adjust(wspace=0.5)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["LMPP"], ax=axs[0][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["Cycling-LP"], ax=axs[0][1], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["LP"], ax=axs[1][0], show = False)

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(9, 15))
plt.subplots_adjust(wspace=0.5)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["PreProB"], ax=axs[0][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["ProB-1"], ax=axs[0][1], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["ProB-2"], ax=axs[1][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["Small-PreB"], ax=axs[1][1], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["Large-PreB"], ax=axs[2][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["IM-B"], ax=axs[2][1], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["ILCP"], ax=axs[3][0], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["NK"], ax=axs[3][1], show = False)
muon.pl.umap(mudata_multiome, color=["anno_wnn_v51"], groups=["T"], ax=axs[4][0], show = False)

## Save

In [ ]:
mudata_multiome["rna"].obsm["X_multiVI"] = mudata_multiome.obsm["X_multiVI"].copy()
mudata_multiome["rna"].obsm["X_umap_multiVI"] = mudata_multiome.obsm["X_umap_multiVI"].copy()

mudata_multiome["rna"].obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"] = mudata_multiome.obs["anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3"].copy()

mudata_multiome["rna"].uns['anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3_colors'] = mudata_multiome.uns['anno_wnn_v51_HSCs-subcluster_leiden_wnn_0.3_colors'].copy()
mudata_multiome["rna"].uns['anno_wnn_v51_colors'] = mudata_multiome.uns['anno_wnn_v51_colors'].copy()
mudata_multiome["rna"].uns['PCW_colors'] = mudata_multiome.uns['PCW_colors'].copy()
mudata_multiome["rna"].uns['libraryID_colors'] = mudata_multiome.uns['libraryID_colors'].copy()

In [ ]:
adata = mudata_multiome["rna"].copy()

In [ ]:
adata.write_h5ad("/work/DevM_analysis/01.annotation/10.integration_joint_clean_MultiVI/data/adata_FL_with-MultiVI.h5ad")